# Imports

In [ ]:
# Import the required libraries 
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

## Processing Dataset

In [ ]:
text_dir = 'D:\\Tu Beo\\Education\\FoodVisor\\data\\UPMC_Food101\\texts_test_txt'

In [ ]:
all_texts = datasets.load_files(text_dir, 
            description=None, categories=None, load_content=True, shuffle=True, 
                                        encoding='utf-8', decode_error='strict', random_state=0)

In [ ]:
all_texts.keys()

In [ ]:
all_texts.filenames[:5]

In [ ]:
all_texts.target_names[:5]

In [ ]:
all_texts.target[:5]

In [ ]:
text_train,text_test , y_train, y_test = train_test_split(
    all_texts.data, all_texts.target, test_size=0.33, random_state=42)

In [ ]:
# Feature Engineering 
print ("TF-IDF on text data ... ")
tfidf = TfidfVectorizer(binary=True)
X_train = tfidf.fit_transform(text_train)
#.astype('float16')
X_test = tfidf.transform(text_test)
#.astype('float16')
print ("Done ! ")

In [ ]:
X_train.shape

In [ ]:
train_size, num_words = X_train.shape
num_classes = len(all_texts.target_names)

## Create Keras model

In [ ]:
# Model Training 
def build_model():
    model = Sequential()
    model.add(Dense(256, input_dim=num_words, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(160, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
print ("Creating model ... ")
estimator = KerasClassifier(build_fn=build_model, epochs=50, batch_size=128)

## Training

In [ ]:
print("Training ...")
estimator.fit(X_train, y_train)
print("Completed !")

## Prediction

In [ ]:
# Predictions 
print ("Predict on test data ... ")
y_pred = estimator.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : ", accuracy)

## Torch model

In [ ]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class myNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(myNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)
        self.dropout = nn.Dropout(0.3) 
        self.softmax = nn.Softmax()

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.output_layer(out)
        out = self.softmax(out)
        return out


net = myNet(num_words, 256, num_classes)


In [ ]:
class myNet2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(myNet2, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, num_classes),
        ) 

    def forward(self, x):
        x = self.classifier(x)
        return x

In [ ]:
net2 = myNet2(num_words, 1024, num_classes)

In [ ]:
print(net2)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=0.1)

### Creating Data generator

In [ ]:
def data_gen(data,labels,batch_size=64,shuffle=True):
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(data))
        data = data[index]
        labels = labels[index]
    for idx in range(0,len(data),batch_size):
        yield(data[idx:idx+batch_size],labels[idx:idx+batch_size])

In [ ]:
def train_model(model,size,conv_feat=None,labels=None,epochs=1,optimizer=None,criterion=None ,train=True,shuffle=True):
    loss_history = []
    acc_history = []
    
    if train:
        model.train()
    else:
        model.eval()
        
    for epoch in range(epochs):
        batches = data_gen(data=conv_feat,labels=labels,shuffle=shuffle)
        total = 0
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in batches:
            if use_gpu:
                inputs , classes = torch.from_numpy(inputs).cuda(), torch.from_numpy(classes).cuda()
            else:
                inputs , classes = Variable(torch.from_numpy(inputs).float()), Variable(torch.from_numpy(classes).long())
                
            inputs = inputs.view(inputs.size(0), -1)
            outputs = model(inputs)
            loss = criterion(outputs,classes)           
            if train:
                if optimizer is None:
                    raise ValueError('Pass optimizer for train mode')
                optimizer = optimizer
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            #print(loss)
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Epoch: {:d} Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch, epoch_loss, epoch_acc))
        
        loss_history.append(epoch_loss)
        acc_history.append(epoch_acc)
        
    return loss_history,acc_history
    

In [ ]:
type(X_train.toarray())

In [ ]:
X_train_torch = torch.from_numpy(X_train.toarray())
y_train_torch = torch.from_numpy(y_train)

In [ ]:
if use_gpu:
    net = net.cuda()

In [ ]:
use_gpu = False

In [ ]:
from torch.autograd import Variable

In [ ]:
%%time
train_model(model=net2,size=train_size,conv_feat=X_train.toarray(),labels=y_train,
            epochs=10,optimizer=optimizer, criterion = criterion,train=True,shuffle=False)

In [ ]:
class OurNet(nn.Module):
    def __init__(self, n_inputs, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(n_inputs,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [ ]:
net3 = OurNet(num_words, 1024, num_classes)

In [ ]:
batch_size = 64
num_epochs = 10
ouroptimizer = optim.SGD(net3.parameters(), lr=0.1)
# Train the Model
for epoch in range(num_epochs):
    batches = data_gen(data=X_train.toarray(),labels=y_train,shuffle=False)
    i = 0
    losses = 0.
    for inputs,classes in batches:
        i += 1
        articles = Variable(torch.FloatTensor(inputs))
        labels = Variable(torch.LongTensor(classes))

        # Forward + Backward + Optimize
        ouroptimizer.zero_grad() # zero the gradient buffer
        outputs = net3(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        ouroptimizer.step()
        
        losses += loss.data.item()

        #print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
 		#		%(epoch+1, num_epochs, i, train_size//batch_size, loss.data.item()))
    print(losses)

## Most informative words

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
clf = LinearSVC()
clf.fit(X_train, y_train)

In [ ]:
print ("Predict on test data ... ")
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : ", accuracy)

In [ ]:
def print_top10(vectorizer, clf, class_labels):
    """
    Prints features with the highest coefficient values, per class \n
    https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers
    """
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("%s: %s" % (class_label,
              ", ".join(feature_names[j] for j in top10)))

In [ ]:
print_top10(tfidf, clf, all_texts.target_names)